<a href="https://colab.research.google.com/github/Lucy-Moctezuma/ML-Tutorial-for-Antibiotic-Resistance-Predictions-for-E.-Coli/blob/main/4)_Extreme_Gradient_Boosted_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(**Note:**
Click on the button that reads *“Open in Colab”* to open this code in Google Colab. Once open in Google Colab, you can make a copy of the notebook in your personal drive and run the code by clicking a little triangle/arrow to the left of each code block.)

# **Extreme Gradient Boosted Tree**

## ***Objectives for this Notebook***
- Expand concepts learned about tree methods (ensemble methods) into Extreme Gradient Boosted Trees Algorithm.
- Create functions to implement Gradient Boosted Trees into Moragadivand's dataset.

**Extreme Gradient Boosted Tree (XGBoost)** is another ensemble method similar to Random Forests as it also uses several Decision Trees, each tree using a random subset of features and observations. XGBoost can use regressor trees or classifier trees as well. But we will only focus on using it for Classification. Below we have an outline of the main differences between Random Forests and XGBoost Trees.

|<font size=4>Random Forest|<font size=4>XGBoost Tree|
|:---|:---|
|<font size=3>All trees are created as a group and are independent|<font size=3>Trees are created sequentially (one tree at a time) <br> and depend on the previous trees residual outputs|
|<font size=3>Uses **bagging**: *Boostrapping* + *Aggregation* <br> - *Boostrapping:* random sampling with replacement<br> - *Aggregation:* adding all trees outputs and majority <br> votes decides final forecast|<font size=3>Uses **boosting** each subsequent tree attempts to <br> create a better predictor than the preceding one, <br>random sampling of observations happen without <br>replacement|
|<font size=3>Individual trees predict target labels|<font size=3>Individual trees predict **residuals**, which is the difference  <br> between the predicted and observed prediction values|

We will later detail how residuals are calculated as we get further into the notebook.

## **XGBoost General Structure**

![xgboost.jpg](https://drive.google.com/uc?export=view&id=12nVLJtdBGBJUp2EMFcoXPBg8c24ewFSg)

Each classifier tree in the XGBoost Model is built differently than the ones for Random Forest, we will see in more detail the different parts of an XGBoost tree later on.

### **1) Importing Packages needed**

Something to note if you decide to check the source documentation is that the import for XGBoost Classifier model uses a different package. Unlike all our previous models, that use "sklearn", this model comes in a separate package named "xgboost".

**NOTE:** Similar code as in previous notebook

In [ ]:
# Data manipulation imports for ML
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype

# Import packages for Gradient-Boosted Tree model and hyperparameter tunning
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedGroupKFold, KFold

# Imports for model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, get_scorer_names
from sklearn.metrics import f1_score, make_scorer, accuracy_score, recall_score, precision_score
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

# Imports for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_tree
from xgboost import plot_importance

# Imports for file management
import os
from google.colab import drive
drive.mount('/content/drive')

### **2) Loading CSV file and creating dataframes for each antibiotic**

Similar to the previous notebook we loaded the dataframe created in the first notebook of this tutorial, then we create a dataframe for each antibiotic. To check that our function works we will test it using the "AMP" antibiotic.

**NOTE:** Similar code as in previous notebook

In [ ]:
# Loads csv file as a dataframe
filepath = '/content/drive/My Drive/EColi_ML_CSV_files/'

# reads csv file as a dataframe
All_Drugs_df = pd.read_csv(filepath+"EColi_Merged_df.csv", na_values="NaN")
All_Drugs_df.head()

### **3) Separating each Drug Dataframe into 4 sections : Training (features and labels) and Testing (features and labels)**

**NOTE:** Similar code as in previous notebook

In [ ]:
# Separating each dataframe into Labels and Features for training and testing data.
# Our function uses the handy train_test_split() function.

def Split_train_test(drug):
  #here we make a list of the columns we want to keep: the column for the isolate, the column for the drug we are interested in and all features (starting from column 13).
  df_list = [All_Drugs_df[["MLST","Isolate",drug,"Year"]], All_Drugs_df.iloc[:,15:]]

  #here we create a data frame with just the columns we wanted to keep.
  Drug_df = pd.concat(df_list, axis=1)

  #here we drop all rows with missing data
  Drug_df = Drug_df.dropna()

  # Creating a dictionary to store each antibiotic datasets
  Train_test_dic = {}

  # Defining the label columns
  labels = Drug_df[drug]

  # Defining features columns
  features = Drug_df.drop(columns=[drug])

  # Separating training (features and labels) and testing (features and labels) datasets
  features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.20, random_state=42, stratify=labels)

  # storing each data chunk in a dictionary
  Train_test_dic['labels_train'] = labels_train
  Train_test_dic['features_train'] = features_train
  Train_test_dic['labels_test'] = labels_test
  Train_test_dic['features_test'] = features_test

  return Train_test_dic

In [ ]:
# Implementing the function Split_train_test() for AMP example
AMP_Train_test_dic = Split_train_test("AMP")

# checking the shape of each dataframe or series stored in the dictionary created for drug AMP
print("AMP")
for k, df in AMP_Train_test_dic.items():
  print(k, df.shape)
  # counting how many of the labels have susceptible versus resistant ones
  if k.startswith("labels"):
    print(df.value_counts())

This time we have more Resistant E.Coli samples than Susceptible ones, unlike in our previous notebook. We will be looking at the AMP antibiotic and **the total number of training observations is 626.**

### **4) Creating different combination of features before training**

**NOTE:** Same as prior notebook code

In [ ]:
# making a list of combinations of data sources we would like to test in our ML models
combo_list = ['Y', 'G', 'GY']

# making a function that creates different feature combinations of the predictor features
def combo_feat(features_df, drug, combo):

  # Isolating Year as a feature and
  year_filter = [col for col in features_df if col.startswith("Year")]
  year_feat = features_df[year_filter]

  # creating Gene precence column filters for features_df
  gene_presc_filter = [col for col in features_df.columns if col not in year_filter and col != "Isolate"]
  gene_presc_feat = features_df[gene_presc_filter]

  if combo == 'Y':
    df_list = [features_df[['MLST','Isolate']], year_feat]
    Y_feat_df = pd.concat(df_list, axis=1)
    Y_feat_df = Y_feat_df.drop(columns=['Isolate'])
    return Y_feat_df

  if combo == 'G':
    df_list = [features_df['Isolate'], gene_presc_feat]
    G_feat_df = pd.concat(df_list, axis=1)
    G_feat_df = G_feat_df.drop(columns=['Isolate'])
    return G_feat_df

  if combo == 'GY':
    df_list = [features_df['Isolate'], year_feat, gene_presc_feat]
    GY_feat_df = pd.concat(df_list, axis=1)
    GY_feat_df = GY_feat_df.drop(columns=['Isolate'])
    return GY_feat_df

In [ ]:
# Implementing combo_feat() function created for training data
AMP_GY_train_df = combo_feat(AMP_Train_test_dic['features_train'],"AMP","GY")

# looking only at the feature column names for the combination for "GY" for drug "AMP" for training data
AMP_GY_train_df.columns

### **5) Creating Gradient Boosted Trees model and training it per feature combination**

Notice that in the creation of an Extreme Gradient Boosted Classifier model within our function, the training labels need to be encoded, that is be transformed from **R** to **0** and **S** to **1**. Our function, will also print the number of trees created.

In [ ]:
# creating Gradient-Boosted Trees model function
def run_GB(feat_train_df, lab_train, drug, combo):
  # creating encoder to transform R and S into 0 and 1 respectively
  labels = lab_train
  le = preprocessing.LabelEncoder()
  le.fit(labels)
  labels_t = le.transform(labels)
  # Creating the XGB Classifier model
  print(drug +" Training combo: "+ combo)
  GB =  XGBClassifier(random_state = 42)
  # Training XGB Classifier model
  feat = feat_train_df.drop(columns=["MLST"])
  GB = GB.fit(feat, labels_t)
  # Checking number of trees in the model
  print("Number of Decicion Trees in XGB Classifier model:", GB.n_estimators)
  return GB

In [ ]:
# implementing run_GB() for specific drug feature combination dataframe
GB_AMP_GY_model = run_GB(AMP_GY_train_df, AMP_Train_test_dic['labels_train'],"AMP","GY")
GB_AMP_GY_model

As you can see there are many parameters we can tune in our XGB classifier. XGBoost is a preferred model by many practitioners because the model considers tuning how the decisions tree are created and trained (e.g. learning_rate and max_leaves). In addition, the model considers computer performance optimization (e.g. n_jobs) when calculating trees.

#### **XGBoost Classification Tree**

Unlike Random Forests, we notice that the XGBoost is made of smaller looking trees, in this case we have a tree with a 6 level depth. You can change and look at the other trees by changing the argument **num_trees**. Also note that one of the main differences is that these trees are not predicting labels, instead they predict residuals. In the picture below, we can focus on just the bottom part of one of the XGBoost Classifier trees and we will pretend that there are 4 residuals in the last intermediate node:

Let's first take a look at the first XGBoost Tree created:

In [ ]:
#Plotting the first XGBoost Tree
plot_tree(GB_AMP_GY_model, num_trees=0)
fig = plt.gcf()
fig.set_size_inches(40, 15)

#### **Hypothetical Branch of a XGBoost Tree**

Notice that unlike the Random Forest classifier trees, XGboost trees have by default only 6 levels (counting from below the root node). But To get a better idea of the differences between xgboost trees and the ones from our prior module, we can focus on just one of the sections output below, the one that starts with the Decision threshold <font face="monospace">urf2 < 1</font>

![XGBoost-branch.jpg](https://drive.google.com/uc?export=view&id=14V84Jbi1Awe-VnhWyo0nbwWdccc-s8GQ)
***
**Feature Threshold** (Example: <font face="monospace">urf2 < 1</font>)

 This is the same threshold explained in our previous Random Forest Notebook, if the boolean statement is TRUE or "Yes" then it goes to the left leaf, if FALSE or "No, missing" then it goes to the right leaf. Meaning observations where this statement is FALSE or are missing would fall into the right branch.
***
**Residuals** (Example: <font face="monospace">[r1,r5,r3,r7]</font>)

Eventhough these are not displayed in the original graphed tree from the code, we can imagine how these residuals are classified so we have an idea of how the algorithm works. For the sake of simplifying the explanation, let's imagine that the node with Decision threshold <font face="monospace">urf2 < 1</font> has 4 residuals, r1 is the residual for observation 1, r5 for observation 5, so on and so forth. In each of the nodes of an xgboost tree, the residuals for each observation are being classified as we descend on the tree, we have r1 and r5 in the left leaf and r3 and r7 in the right leaf.
***
**Leaf** (Example: <font face="monospace">Leaf=0.338359058</font>)

For XGboost trees, the leaves do not output the number of observations in each class, instead it displays a decimal, which is known as the **Output value**. This output is computed using only the residuals from the final leaf nodes, which in our pseudoexample would be r1 and r5.  Below its the actual formula to calculate it:

$$Output =\frac{\sum Residuals}{\sum[(Previous \ Probability)*(1-Previous \ Probability)] + \lambda}$$

- *$Residuals$* = The diference between the observed (0 for Resistance and 1 for Susceptibility) and predicted probabilities.

- *$Previous \ Probability$* = These are the probabilities predicted using the prior tree. Since this is an example from the first tree created, the default value for it is 0.5. This number will change as we keep building other trees.

- *$\lambda$*= Is the regularization term, the higher the number the lower the output value. This helps us to not follow too closely any particular observation so that our model generalizes well to new data (bias-variance trade-off). The default is set to 1. There are other regularization coeficcients, in this example *$\lambda$* is equivalent to L2 regularization or Rigde Regression. Here is a [link](https://albertum.medium.com/l1-l2-regularization-in-xgboost-regression-7b2db08a59e0#:~:text=Therefore%2C%20the%20L1%20regularization%20decreases,lambda%20term%20increases%20the%20denominator.) for further mathematical explanation if interested.
***

Behind the simple looking XGboost tree there are different metrics used to decide each of the nodes and branches that shape the classifier tree. You can take a peak under the hood of this XGboost tree with the code below and check all the **Gain** and **Cover** scores at each split from the tree we graphed. We will briefly go over these concepts:





In [ ]:
# looking at the metrics for each node of the XGboost classifier Trees
print(GB_AMP_GY_model.get_booster().get_dump(with_stats = True)[0])

#### **OUTPUT INTERPRETATION of get_dump() for one xgboost Tree**
To interpret each **(root and intermediate)** nodes output, we can take look at the following structure:

<font color="gray">`Parent_node #:[feature threshhold] yes=left_child_#, no=right_child_#, missing=left_child_#, gain=gain, cover=cover`

To interpret each **leaf** nodes output, we can examine the below structure:

<font color="gray">`leaf_node_#:leaf=output_value, cover=cover`
***
For each node in our tree, we calculate the following:

- **Similarity Scores (SS):** The scores indicate how similar are a particular grouping of residuals. This score is important because similar residuals imply similarity in observations, which helps with their classification. Below we can see the formula for SS:

$$\frac{(\sum Residuals)^2}{\sum[(Previous \ Probability)*(1-Previous \ Probability)] + \lambda}$$

<a name="gain-def"></a>
- **Gain:** This is calculated for each of the intermediate nodes and measures the relative contribution of the corresponding feature to the model. The higher this value the better the feature is at making a prediction. Below is the formula:

$$Gain = SS_{(Left)} + SS_{(Right)} - SS_{(Parent \ Node)}$$

- **Cover:** is a metric to measure the number of observations affected by the split. This is essentially if the first part of the denominator in the similarity score:

$$Cover = \sum[(Previous \ Probability)*(1-Previous \ Probability)]$$
***
**NOTE:** When researching this model it's important to know that XGBoost $\neq$ Gradient Boosted Tree. The Reason it has the **Extreme** denomination is that XGBoost builds on Gradient Boosted Tree in that it add different methods of Regularization (to help with generalization of the model and combat overfitting) and parallel processing (increase computing efficiency).


### **6) Making predictions from XGBoost model**

**XGBoost Final Decision Made**

To understand how the final decision is made by the entire XGBoost model, we need to revisit again the equations we learned on the very first ML model we used **Logistic Regression**. Because XGBoost Classifier models uses them as well!

**Logit Equation:** converts probabilities into log-odds predictions.
$$ ln(\frac{P}{1-P}) = \hat \beta_0 + \hat \beta_jX$$

**Logistic Regression Equation:** converts log-odds predictions into probability predictions.
$$P = \frac{e^{\hat{\beta}_{0}+\hat{\beta}_{j}X}}{1+e^{\hat{\beta}_{0}+\hat{\beta}_{j}X}}$$

Turns out these Equations are the ones that enables the XGboost Classifier to convert the predictions made by each of these XGboost Classifier trees into probabilities that will ultimately help us classify our observations into **Resistant (R)** or **Susceptible (S)**. Below we can see a graph of how it works:

![final_pred.jpg](https://drive.google.com/uc?export=view&id=1RbLJnuLNpKsNMCF2gv9YLWZYYaXcmPwQ)

Iteration 1:
**Predicted log (odds1) = Initial Log(odds) + LR * Leaf Output that observation falls into tree 1**

Iteration 2:
**Predicted log (odds2) = Predicted Log(odds1) + LR * Leaf Output that observation falls into tree 2**

ETC...

At the end after we iterate to all xgboost trees (in this case 100 times), at each iteration a new tree is added and each time the predicted log (odds) gets updated with a better aproximation until we get a final **Predicted log (odds100)**, which takes in consideration a total of 100 trees. This final prediction is converted to a probability using the logit and logistic regression equations. This should give us the final prediction of **Resistant (R) if probability < 0.5** or **Susceptible (S) if probability > 0.5**

**NOTE:** The Learning Rate in XGBoost is also called **"eta"** in other sources.
***

Below we can run a short piece of code to check the improvements for every 10 iterations out of a total of 100 to test how an XGBoost model works, we are not actually using our own model, the code creates automatically a training and validation sets, from the training data:

In [ ]:
# creates a copy of the labels
num_labels = AMP_Train_test_dic['labels_train'].copy()

# transforms all labels of S into 1 and all labels of R into 0
num_labels.loc[num_labels == "S"] = 1.0
num_labels.loc[num_labels == "R"] = 0.0

# create a matrix using the only the training data and the training labels coded as numbers
feat_df = AMP_GY_train_df.drop(columns=["MLST"])
d = xgb.DMatrix(feat_df, label=num_labels)

# set up basic parameters that are similar to our model
p = {'max_depth':6, 'eta':0.33, 'objective':'binary:logistic'}

# prints results for every 10 iterations out of 100
res = xgb.cv(params = p, dtrain=d, num_boost_round=100, verbose_eval=10, metrics="error")

The code above shows us that the overall trend is that the error rate decreases as number of iteration increases. The outputs are composed of 2 numbers in the following format:

`train-error: #1 + #2     test-error: #1 + #2`

- **#1** is the error shown for training and testing sets, it is essentially (1- accuracy) for every 10th iteration.
- **#2** is the standard deviation of the error rates.


Now that we have an idea of how the XGBoost tree trains and makes its final predictions using all the trees. We can now make predictions with it, below a simple function is created similar to the code in the previous notebook:

In [ ]:
def predict(GB_combo_Model, features_test):
  feat = features_test.drop(columns=['MLST'])
  labels_pred = GB_combo_Model.predict(feat)
  if is_numeric_dtype(labels_pred):
    # tranforming labels from numbers to letters
    lab_pred_t = labels_pred.astype('O')
    lab_pred_t[lab_pred_t==0] = 'R'
    lab_pred_t[lab_pred_t==1] = 'S'
    return lab_pred_t
  else:
    return labels_pred

In [ ]:
# Implementing combo_feat() function created for testing data
AMP_GY_test_df = combo_feat(AMP_Train_test_dic['features_test'],"AMP","GY")

# looking only at the feature column names for the combination for "GY" for drug "AMP" for testing data
AMP_GY_test_df.shape

In [ ]:
# Implementation of the predict() function using the feature combination "GS"
AMP_GY_labels_pred = predict(GB_AMP_GY_model,AMP_GY_test_df)

# observe how many predictions were made for each category "R" and "S"
print("Labels predicted: ", np.unique(AMP_GY_labels_pred, return_counts=True))

Our **XGBoost model** has predicted 118 Resistant observations and 39 Susceptible observations.

### **7) Evaluating our model using a confusion matrix and metrics**

Similarly to the previous notebook, we will evaluate our XGBoost model by using a Confusion Matrix and respective metrics. Below is a quick review of these, remember that there is one Accuracy score, but Recall and Presicion should have as many sets as classes our model its trained to predict:

|<font size=3>Metrics|<font size=3>General formula| <font size=3>Formula for 2 classes|
|--|:-:|:-:|
|<font size=3>**Accuracy**|<font size=3>$\frac{Correctly \ classified}{All \ Predicted}$|<font size=3>$\frac{TP + TN}{TP + TN + FN + FP}$|
|<font size=3>**R Recall:**|<font size=3>$\frac{Correctly \ classified \ as \ R}{All \ Actual \ R}$|<font size=3>$\frac{TP}{TP + FN}$|
|<font size=3>**R Precision:**|<font size=3>$\frac{Correctly \ classified \ as \ R}{All \ Predicted \ R}$|<font size=3>$\frac{TP}{TP + FP}$|

**NOTE:** Code below is the same as in previous notebook


In [ ]:
# Creating a function that evaluates our model using our actual and predicted data
def evaluate(GB_combo_model, labels_test, labels_pred, cf= True, show_results=True):
  report = classification_report(labels_test, labels_pred, output_dict = True)
  if cf == True:
    cm = confusion_matrix(labels_test, labels_pred, labels=np.where(GB_combo_model.classes_<1,"R","S"))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.where(GB_combo_model.classes_<1,"R","S"))
    disp.plot()
    plt.grid(False)
    plt.show()
  if show_results == True:
    print("Results")
    print('Accuracy:',report['accuracy'])
    print('R recall:',report['R']['recall'])
    print('S recall:',report['S']['recall'])
    print('R precision:',report['R']['precision'])
    print('S precision:',report['S']['precision'])
  return [report['accuracy'], report['R']['recall'], report['S']['recall'], report['R']['precision'], report['S']['precision']]

In [ ]:
# implementing the evaluate() function
Model_Report = evaluate(GB_AMP_GY_model, AMP_Train_test_dic['labels_test'],AMP_GY_labels_pred)

**b) Create a feature importance graph showing which features were the most important**

There are actually different ways to evaluate the importance of features in this model, some are based on the **Cover**, others in **Gain**, etc. Since we are already familiar with how these are calculated in [Part 5](#gain-def).

The plot below will focus on the **Gain**. Since every node in every tree of our XGBoost model have their own gain, the value shown is just the average of each node. The Gain is the most relevant attribute to interpret the relative importance of each feature.

In [ ]:
# Plotting feature importance ("gain")
feat_imp = GB_AMP_GY_model.get_booster().get_score(importance_type="gain")
for k in feat_imp.keys():
    feat_imp[k] = round(feat_imp[k],2)

plot_importance(feat_imp, max_num_features = 20, importance_type = "gain", show_values=True, height=0.7)
plt.savefig('/content/drive/My Drive/EColi_ML_Plots/GB_AMP_GY_feat_importance.jpg',dpi=400, bbox_inches="tight")

Below we can see that the most important feature in the model **GB_AMP_GY_model**, based in **Gain** metric is the gene `bla`.
The Gain implies the relative contribution of the feature to the model. It takes each feature's contribution for each tree in the model. A higher value of this metric when compared to another feature implies it is more important for generating a prediction.

### **8) (OPTIONAL) Hyperparameter Tunning and Crossvalidation for XGBoost model**

This section of the notebook is optional.
Here we will carry out hyperparameter tuning using cross-validation to optimize the model's performance and ensure its robustness. We will also include code to use a blocked cross-validation design. These approaches are important to make good ML models. However, if you are just here to learn the basics, feel free to skip section 8.

#### **a) Hyperparameter Tunning**

For now we will just perform the hyperparameter tunning for "eta" (seen in part 6) which ranges from 0 to 1, which is the learning rate for each xgboost tree.
If you want to tune other hyperparameters in your model feel free to read the [documentation](https://xgboost.readthedocs.io/en/stable/parameter.html) for each of them.

As usual we start by creating a dictionary with values to test:


In [ ]:
# performing hyperparameter tunning for model
## Creating dictionary of parameters to tune.
hparam = {"eta":[0.1, 0.2, 0.4, 0.8]}

#### **b) Crossvalidation**

In [ ]:
# Creating function to perform hyper parameter tunning of model
def GB_hp_tune(param, feat_train_df, lab_train, v=3, cv=4):
  # Creating XGBOOST model
  model = XGBClassifier(random_state = 42)
  # performing the hyper parameter tunning using crossvalidation
  scoring_dic = {'f1_macro':make_scorer(f1_score , average='macro')}
  # isolating the features used for training
  feat = feat_train_df.drop(columns=["MLST"])
  # tranforming labels from letters to 0 & 1
  lab_train_t = lab_train.replace({'R': 0, 'S': 1})
  # Fitting using grid search parameters depending on crossvalidation scheme used
  if str(cv).isnumeric():
    cv = KFold(cv)
    gs = GridSearchCV(model, param, scoring=scoring_dic,cv=cv, refit='f1_macro', verbose=v, return_train_score=True)
    gs.fit(feat, lab_train_t)
  elif cv == "blocked":
    groups= feat_train_df['MLST']
    cv = StratifiedGroupKFold(n_splits=4)
    gs = GridSearchCV(model, param, scoring=scoring_dic,cv=cv, refit='f1_macro', verbose=v, return_train_score=True)
    gs.fit(feat, lab_train_t, groups=groups)
  else:
    print("Please provide a valid crossvalidation scheme `blocked` or an integer")
  print(gs.best_params_)
  print(gs.best_score_)
  return gs.best_estimator_

In [ ]:
# Running the function created (takes longer time)
GB_tunned = GB_hp_tune(hparam, AMP_GY_train_df, AMP_Train_test_dic['labels_train'])

The best xgboot model for AMP antibiotic using Year and Gene Presence/Absence features was obtained using eta = 0.2, which produced an f1_macro score of 0.9014

### **9) Use all functions and evaluate every drug in every feature combination!**
**NOTE:** Code below is similar as in previous notebook

#### **a) Lets recall the list of drugs we have available and the combination of features we are interested in**

In [ ]:
# let's check all drugs
drug_list = All_Drugs_df.iloc[:,3:15].columns
print(drug_list)

# let's see all combinations we are interested in
print(combo_list)

#### **b) Create a loop that will go through all our functions using the lists above**

In [ ]:
for fname in os.listdir(filepath):
    if fname.endswith('GB_metrics_df.csv'):
        print("A csv with stored results for XGBoost has already been created. Please check your Google Drive directory.")
        break
else:
  # Lets use all our functions this time and save our report into a single data structure
  GB_model_metrics = {}

  for drug in drug_list:
    print(drug)
    # splits each drug df into a dictionary with testing and training data
    Test_Train_dic = Split_train_test(drug)
    for combo in combo_list:
      # Training each drug and combo features
      labels_train = Test_Train_dic["labels_train"]
      # create corresponding feature_df for training
      features_train = combo_feat(Test_Train_dic["features_train"], drug, combo)
      print(drug+"_"+combo)

      # runs xgboost model using the corresponding training feature_df
      GB_combo_model = run_GB(features_train, labels_train, drug, combo)

      #Optional: If you want to use the hyperparamter tuning step 8 use this line instead of the previous one:
      #GB_combo_model = GB_hp_tune(hparam, features_train, labels_train, v=0)

      # create corresponding feature_df for testing
      features_test = combo_feat(Test_Train_dic["features_test"], drug, combo)
      # generate predictions based on the feature combination tested
      labels_pred = predict(GB_combo_model, features_test)
      # loading the actual labels
      labels_test = Test_Train_dic["labels_test"]
      # Evaluating our model
      report = evaluate(GB_combo_model, labels_test, labels_pred, cf=False, show_results=False)
      # Saving the results into a dictionary
      GB_model_metrics[drug+"_"+combo] = report
      print(report)
  # convert dictionary into a dataframe
  GB_metrics = pd.DataFrame.from_dict(GB_model_metrics, orient='index',columns=["Accuracy", "R_recall", "S_recall", "R_precision", "S_precision"]).reset_index()
  GB_metrics = GB_metrics.rename(columns = {'index':'Drug_combo'})

  # saving our metric results into a CSV file
  GB_metrics.to_csv(filepath+"GB_metrics_df.csv", index= False)

### **10) Create a bar graph showing metrics for all drugs using only GY features**

Using the CSV we created above with all our results, we will create a bargraph to visualize metrics for every antibiotic that uses the GY feature (Gene Presence/Absence and Year). In the graph below we are comparing Accuracy and Precision scores for both classes.

The code below serves to create a new directory to store all our Machine Learning plots and figures created. If you run the code more than once it will let you know that the figure has already been created in the directory.

In [ ]:
try:
  # makes a directory for all your plot images
  os.mkdir('/content/drive/My Drive/EColi_ML_Plots')
except:
  print("A directory was already created to store your plot")

# filtering for all the rows that contain GY features
GB_metrics = pd.read_csv("/content/drive/MyDrive/EColi_ML_CSV_files/GB_metrics_df.csv")
GY_filter = [drug_combo for drug_combo in GB_metrics['Drug_combo'] if drug_combo.endswith("GY")]
GY_df = GB_metrics.loc[GB_metrics["Drug_combo"].isin(GY_filter)]

# Figure Size
sns.set_theme(style="whitegrid")
fig = plt.figure(figsize =(15, 6))

# Adding title
plt.title('Accuracy, R_precision, S_precision', fontsize = 12)

# Variables to be plotted
x = np.arange(len(GY_df["Drug_combo"]))
acc = list(GY_df["Accuracy"])
R_prec = list(GY_df["R_precision"])
S_prec = list(GY_df["S_precision"])

# Plotting barcharts
acc_bar=plt.bar(x-0.25, height= acc, width=0.25, color="grey", edgecolor="gray")
rprec_bar=plt.bar(x, height= R_prec, width=0.25, color="goldenrod", edgecolor="gray")
sprec_bar=plt.bar(x+0.25, height= S_prec, width=0.25, color="khaki", edgecolor="gray")

plt.xticks([r for r in range(len(GY_df["Drug_combo"]))],
            GY_df["Drug_combo"], fontsize = 12)

#legend
fig.legend([acc_bar,rprec_bar,sprec_bar],["Accuracy", "R_precision", "S_precision"], bbox_to_anchor=(0.4,-0.35, 0.04, 0.4), fontsize=12)

# Saving bargraph into our new directory
plt.savefig('/content/drive/My Drive/EColi_ML_Plots/GB_GY_Accuracy_and_Recall_Scores.jpg',dpi=400, bbox_inches="tight")

# Show Plot
plt.show()

We have finally created and saved all the XGBoost Classifier results for every antibiotic and combination of features we were interested in. Next we will be moving away from tree-based or ensemble methods and learn about our last Machine Learning Method: [Neural Networks](https://colab.research.google.com/drive/1ytPswhUtQjNzRmJbZaFVw4_SGX5wG7iK?usp=drive_link).